# Retrieving derivative works from Wikidata

Note that in this notebook works can be of any kind. This means that we are not dealing with particular creative works or works of art. In fact, in the result every work authored by someone can be found, from buildings to poems.

In [1]:
from pandas import *
from queryWikidata import query_wikidata

In [2]:
endpoint = "https://query.wikidata.org/sparql"
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:105.0) Gecko/20100101 Firefox/105.0"

## 1) Count all the derivative works present on Wikidata

To do that, we rely on the following properties:
- P737 influenced by
- P144 based on
- P941 inspired by
- P8371 references work
- P4969 derivative work
- P58 screenwriter
- P1877 after a work by
- P674 characters
- P170 creator
- P50 author



### 1.1 Properties based definition of works and derivative works

Works and derivative works are defined only on the basis of the relationship occurring between them.

In [3]:
q1_derivative_works = """
SELECT (COUNT(DISTINCT ?derivativeWork)AS ?derivativeWorksCount)
WHERE {
    {
      VALUES ?p {wdt:P737 wdt:P144 wdt:P941 wdt:P8371 wdt:P4969} 
      ?derivativeWork ?p ?work .
    }UNION{
      ?derivativeWork wdt:P1877 ?creator .
    }
}
"""
derivative_works_df1 = query_wikidata(endpoint, q1_derivative_works, user_agent, True)
derivative_works_df1

,derivativeWorksCount
0,87362


### 1.2 Derivative works with works having an author or creator
The only distinction from the previous query is that here it is specified that the "works" must have an author or a creator. Note that we are not explicitly referring to a person. In fact authors and creators can be also corporations, groups of people (e.g., bands), etc.

In [4]:
q2_derivative_works = """
SELECT (COUNT(DISTINCT ?derivativeWork)AS ?derivativeWorksCount)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                       
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    
    {
      VALUES ?p {wdt:P737 wdt:P144 wdt:P941 wdt:P8371 wdt:P4969} 
      ?derivativeWork ?p ?work .
    }UNION{
      ?derivativeWork wdt:P1877 ?creator .
    }
}
"""
derivative_works_df2 = query_wikidata(endpoint, q2_derivative_works, user_agent, True)
derivative_works_df2

,derivativeWorksCount
0,44805


### 1.3 Derivative works with derivative works having an author or creator
In this query the only change is that the previous restriction applies to the derivative works, not to the works.

In [5]:
q3_derivative_works = """
SELECT (COUNT(DISTINCT ?derivativeWork)AS ?derivativeWorksCount)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator .
     }UNION{
      ?derivativeWork wdt:P50 ?creator .                       
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?derivativeWork .
    }
    
    {
      VALUES ?p {wdt:P737 wdt:P144 wdt:P941 wdt:P8371 wdt:P4969} 
      ?derivativeWork ?p ?work .
    }UNION{
      ?derivativeWork wdt:P1877 ?creator .
    }
}
"""
derivative_works_df3 = query_wikidata(endpoint, q3_derivative_works, user_agent, True)
derivative_works_df3

,derivativeWorksCount
0,23913


### 1.4 Derivative works with both works and derivative works having an author or a creator
In this query, both works and cretive works must have an author or a creator to be included in the results.

In [6]:
q4_derivative_works = """
SELECT (COUNT(DISTINCT ?derivativeWork)AS ?derivativeWorksCount)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                       
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                       
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }
    
    {
      VALUES ?p {wdt:P737 wdt:P144 wdt:P941 wdt:P8371 wdt:P4969} 
      ?derivativeWork ?p ?work .
    }UNION{
      ?derivativeWork wdt:P1877 ?creator .
    }
}
"""
derivative_works_df4 = query_wikidata(endpoint, q4_derivative_works, user_agent, True)
derivative_works_df4

,derivativeWorksCount
0,18518


## 2) Analyse the use of single properties to connect a product with its derivative work

### 2.1 Influenced by (`P737`)

#### 2.1.1 Properties based definition of works and derivative works

In [7]:
q1_influenced_by = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInfluencedBy)
WHERE {

    ?derivativeWork wdt:P737 ?work . 
                              
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
influenced_by_df1 = query_wikidata(endpoint, q1_influenced_by, user_agent, True)
influenced_by_df1

,work,derivativeWorkInfluencedBy
0,10597,10307


#### 2.1.2 Derivative works with works having an author or creator

In [8]:
q2_influenced_by = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInfluencedBy)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?derivativeWork wdt:P737 ?work . 
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
influenced_by_df2 = query_wikidata(endpoint, q2_influenced_by, user_agent, True)
influenced_by_df2

,work,derivativeWorkInfluencedBy
0,443,757


#### 2.1.3 Derivative works with derivative works having an author or creator

In [9]:
q3_influenced_by = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInfluencedBy)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator .
     }UNION{
      ?derivativeWork wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P737 ?work . 
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
influenced_by_df3 = query_wikidata(endpoint, q3_influenced_by, user_agent, True)
influenced_by_df3

,work,derivativeWorkInfluencedBy
0,542,329


#### 2.1.4 Derivative works with both works and derivative works having an author or a creator

In [10]:
q4_influenced_by = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInfluencedBy)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P737 ?work . 
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
"""
influenced_by_df4 = query_wikidata(endpoint, q4_influenced_by, user_agent, True)
influenced_by_df4

,work,derivativeWorkInfluencedBy
0,186,174


### 2.2 Based on (`P144`)

#### 2.2.1 Properties based definition of works and derivative works

In [11]:
q1_based_on = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkBasedOn)
WHERE {

    ?derivativeWork wdt:P144 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

based_on_df1 = query_wikidata(endpoint, q1_based_on, user_agent, True)
based_on_df1

,work,derivativeWorkBasedOn
0,35093,53158


#### 2.2.2 Derivative works with works having an author or creator

In [12]:
q2_based_on = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkBasedOn)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?derivativeWork wdt:P144 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

based_on_df2 = query_wikidata(endpoint, q2_based_on, user_agent, True)
based_on_df2

,work,derivativeWorkBasedOn
0,19003,32456


#### 2.2.3 Derivative works with derivative works having an author or creator

In [13]:
q3_based_on = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkBasedOn)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator .
     }UNION{
      ?derivativeWork wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P144 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

based_on_df3 = query_wikidata(endpoint, q3_based_on, user_agent, True)
based_on_df3

,work,derivativeWorkBasedOn
0,9562,12816


#### 2.2.4 Derivative works with both works and derivative works having an author or a creator

In [14]:
q4_based_on = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkBasedOn)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P144 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

based_on_df4 = query_wikidata(endpoint, q4_based_on, user_agent, True)
based_on_df4

,work,derivativeWorkBasedOn
0,7577,10134


### 2.3 Inspired By (`P941`)

#### 2.3.1 Properties based definition of works and derivative works

In [15]:
q1_inspired_by = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInspiredBy)
WHERE {

    ?derivativeWork wdt:P941 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

inspired_by_df1 = query_wikidata(endpoint, q1_inspired_by, user_agent, True)
inspired_by_df1

,work,derivativeWorkInspiredBy
0,5509,5351


#### 2.3.2 Derivative works with works having an author or creator

In [16]:
q2_inspired_by = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInspiredBy)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?derivativeWork wdt:P941 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

inspired_by_df2 = query_wikidata(endpoint, q2_inspired_by, user_agent, True)
inspired_by_df2

,work,derivativeWorkInspiredBy
0,1611,1777


#### 2.3.3 Derivative works with derivative works having an author or creator

In [17]:
q3_inspired_by = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInspiredBy)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator .
     }UNION{
      ?derivativeWork wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P941 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

inspired_by_df3 = query_wikidata(endpoint, q3_inspired_by, user_agent, True)
inspired_by_df3

,work,derivativeWorkInspiredBy
0,2298,2160


#### 2.3.4 Derivative works with both works and derivative works having an author or a creator

In [18]:
q4_inspired_by = """
SELECT  (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkInspiredBy)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P941 ?work .
                              
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

inspired_by_df4 = query_wikidata(endpoint, q4_inspired_by, user_agent, True)
inspired_by_df4

,work,derivativeWorkInspiredBy
0,897,961


### 2.4 References Work (`P8371`)

#### 2.4.1 Properties based definition of works and derivative works

In [19]:
q1_references_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkReferencesWork)
WHERE {

    ?derivativeWork wdt:P8371 ?work .
                      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

references_work_df1 = query_wikidata(endpoint, q1_references_work, user_agent, True)
references_work_df1

,work,derivativeWorkReferencesWork
0,1254,597


#### 2.4.2 Derivative works with works having an author or creator

In [20]:
q2_references_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkReferencesWork)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?derivativeWork wdt:P8371 ?work .
                      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

references_work_df2 = query_wikidata(endpoint, q2_references_work, user_agent, True)
references_work_df2

,work,derivativeWorkReferencesWork
0,609,368


#### 2.4.3 Derivative works with derivative works having an author or creator

In [21]:
q3_references_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkReferencesWork)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator .
     }UNION{
      ?derivativeWork wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P8371 ?work .
                      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

references_work_df3 = query_wikidata(endpoint, q3_references_work, user_agent, True)
references_work_df3

,work,derivativeWorkReferencesWork
0,905,254


#### 2.4.4 Derivative works with both works and derivative works having an author or a creator

In [22]:
q4_references_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkReferencesWork)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P8371 ?work .
                      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

references_work_df4 = query_wikidata(endpoint, q4_references_work, user_agent, True)
references_work_df4

,work,derivativeWorkReferencesWork
0,498,188


### 2.5 Derivative Work (`P4969`)

#### 2.5.1 Properties based definition of works and derivative works

In [23]:
q1_derivative_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?work wdt:P4969 ?derivativeWork .

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

derivative_work_df1 = query_wikidata(endpoint, q1_derivative_work, user_agent, True)
derivative_work_df1

,work,derivativeWork
0,9378,16142


#### 2.5.2 Derivative works with works having an author or creator

In [24]:
q2_derivative_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?work wdt:P4969 ?derivativeWork .

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

derivative_work_df2 = query_wikidata(endpoint, q2_derivative_work, user_agent, True)
derivative_work_df2

,work,derivativeWork
0,9378,16142


#### 2.5.3 Derivative works with derivative works having an author or creator

In [25]:
q4_derivative_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?work wdt:P4969 ?derivativeWork .

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

derivative_work_df4 = query_wikidata(endpoint, q4_derivative_work, user_agent, True)
derivative_work_df4

,work,derivativeWork
0,3901,5150


#### 2.5.4 Derivative works with both works and derivative works having an author or a creator

In [26]:
q3_derivative_work = """
SELECT (COUNT(DISTINCT ?work) AS ?work) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWork)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator .
     }UNION{
      ?derivativeWork wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?derivativeWork .
    }

    ?work wdt:P4969 ?derivativeWork .

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

derivative_work_df3 = query_wikidata(endpoint, q3_derivative_work, user_agent, True)
derivative_work_df3

,work,derivativeWork
0,4433,5785


### 2.6 After a Work By (`P1877`)

#### 2.6.1 Properties based definition of works and derivative works

In [27]:
q1_after_work_by = """
SELECT (COUNT(DISTINCT ?creator) AS ?creatorCount) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkAfterWorkBy)
WHERE {

    ?derivativeWork wdt:P1877 ?creator .
      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

after_work_by_df1 = query_wikidata(endpoint, q1_after_work_by, user_agent, True)
after_work_by_df1

,creatorCount,derivativeWorkAfterWorkBy
0,4849,12251


#### 2.6.2 Derivative works with works having an author or creator

In [28]:
q2_after_work_by = """
SELECT (COUNT(DISTINCT ?creator) AS ?creatorCount) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkAfterWorkBy)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }

    ?derivativeWork wdt:P1877 ?creator .
      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

after_work_by_df2 = query_wikidata(endpoint, q2_after_work_by, user_agent, True)
after_work_by_df2

,creatorCount,derivativeWorkAfterWorkBy
0,3913,10957


#### 2.6.3 Derivative works with derivative works having an author or creator

In [29]:
q3_after_work_by = """
SELECT (COUNT(DISTINCT ?creator) AS ?creatorCount) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkAfterWorkBy)
WHERE {
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P1877 ?creator .
      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

after_work_by_df3 = query_wikidata(endpoint, q3_after_work_by, user_agent, True)
after_work_by_df3

,creatorCount,derivativeWorkAfterWorkBy
0,1692,5116


#### 2.6.4 Derivative works with both works and derivative works having an author or a creator

In [30]:
q4_after_work_by = """
SELECT (COUNT(DISTINCT ?creator) AS ?creatorCount) (COUNT(DISTINCT ?derivativeWork) AS ?derivativeWorkAfterWorkBy)
WHERE {
    {
      ?work wdt:P170 ?creator .
     }UNION{
      ?work wdt:P50 ?creator .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator ?authorOf ?work .
    }
    {
      ?derivativeWork wdt:P170 ?creator2 .
     }UNION{
      ?derivativeWork wdt:P50 ?creator2 .                           
     }UNION{
      VALUES ?authorOf {wdt:P1455 wdt:P800}      
      ?creator2 ?authorOf ?derivativeWork .
    }

    ?derivativeWork wdt:P1877 ?creator .
      
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} """

after_work_by_df4 = query_wikidata(endpoint, q4_after_work_by, user_agent, True)
after_work_by_df4

,creatorCount,derivativeWorkAfterWorkBy
0,1409,4545
